# Robustness Comparison: Camargo LSTM vs Normal Model

This notebook compares robustness metrics between the Camargo LSTM model and the Normal (DropoutUncertainty) model on the Helpdesk dataset.


In [1]:
import sys
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.insert(0, '..')
sys.path.insert(0, '../..')

from robustness_metrics import (
    load_results, prepare_robustness_results, calculate_aggregate_metrics
)

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 11


In [6]:
# Load Camargo LSTM results
#camargo_results_path = '../../evaluation_results/robustness/sepsis/last_event_attack_2/robustness_results.pkl'
camargo_results_path = '../../evaluation_results/robustness/sepsis/random_event_attack/robustness_results.pkl'
camargo_results = load_results(camargo_results_path)
camargo_results = prepare_robustness_results(camargo_results, save_path=camargo_results_path)
camargo_data = calculate_aggregate_metrics(camargo_results)

print(f"Camargo-LSTM: Loaded {len(camargo_results)} results")


Camargo-LSTM: Loaded 1550 results


In [5]:
# Load UED-LSTM results
#U_ED_LSTM_results_path = '../../evaluation_results/robustness/sepsis/last_event_attack_2/robustness_results.pkl'
#U_ED_LSTM_results_path = '../evaluation_results/robustness/Helpdesk/large_perturbations/robustness_results.pkl'
U_ED_LSTM_results_path = '../../evaluation_results/robustness/sepsis/random_event_attack/robustness_results.pkl'
U_ED_LSTM_results = load_results(U_ED_LSTM_results_path)
U_ED_LSTM_results = prepare_robustness_results(U_ED_LSTM_results, save_path=U_ED_LSTM_results_path)
normal_data = calculate_aggregate_metrics(U_ED_LSTM_results)

print(f"U-ED-LSTM: Loaded {len(U_ED_LSTM_results)} results")

U-ED-LSTM: Loaded 1550 results


In [7]:
# Compare mean_suffix (mean_prediction) of perturbed and clean for first 100 results
import json
import time
import os

# #region agent log
log_path = '/home/chair/henryks_students/leon_urny/.cursor/debug.log'
def log_debug(session_id, run_id, hypothesis_id, location, message, data):
    try:
        # Ensure directory exists
        os.makedirs(os.path.dirname(log_path), exist_ok=True)
        with open(log_path, 'a') as f:
            f.write(json.dumps({
                'sessionId': session_id,
                'runId': run_id,
                'hypothesisId': hypothesis_id,
                'location': location,
                'message': message,
                'data': data,
                'timestamp': int(time.time() * 1000)
            }) + '\n')
            f.flush()  # Ensure data is written immediately
    except Exception as e:
        print(f"DEBUG LOG ERROR: {e}")  # Print error for debugging
# #endregion

# Test logging
log_debug('debug-session', 'run1', 'TEST', 'compare_robustness_models.ipynb:init', 'Logging initialized', {'test': True})

print("="*80)
print("COMPARING MEAN PREDICTIONS: CLEAN vs PERTURBED (First 100)")
print("="*80)

# Get first 100 entries
first_100_items = list(U_ED_LSTM_results.items())[:100]

# DEBUG: Inspect the first entry to understand the structure
if first_100_items:
    first_key, first_entry = first_100_items[0]
    print(f"\nDEBUG: Inspecting first entry: {first_key}")
    print(f"DEBUG: Entry keys: {first_entry.keys()}")
    
    # #region agent log
    log_debug('debug-session', 'run1', 'A', 'compare_robustness_models.ipynb:15', 'Inspecting first entry', {'key': str(first_key), 'entry_keys': list(first_entry.keys())})
    # #endregion
    
    if 'original' in first_entry:
        orig_tuple = first_entry['original']
        print(f"DEBUG: Original tuple length: {len(orig_tuple)}")
        
        # Check prefix structure (Hypothesis C)
        prefix_orig = orig_tuple[0]
        # #region agent log
        log_debug('debug-session', 'run1', 'C', 'compare_robustness_models.ipynb:22', 'Checking prefix keys', {
            'prefix_type': str(type(prefix_orig)),
            'prefix_len': len(prefix_orig) if prefix_orig else None,
            'first_prefix_event_keys': list(prefix_orig[0].keys()) if prefix_orig and len(prefix_orig) > 0 and isinstance(prefix_orig[0], dict) else None,
            'has_Activity': 'Activity' in prefix_orig[0] if prefix_orig and len(prefix_orig) > 0 and isinstance(prefix_orig[0], dict) else False
        })
        # #endregion
        
        # Check suffix structure (ground truth)
        suffix_orig = orig_tuple[1]
        # #region agent log
        log_debug('debug-session', 'run1', 'C', 'compare_robustness_models.ipynb:30', 'Checking suffix keys', {
            'suffix_type': str(type(suffix_orig)),
            'suffix_len': len(suffix_orig) if suffix_orig else None,
            'first_suffix_event_keys': list(suffix_orig[0].keys()) if suffix_orig and len(suffix_orig) > 0 and isinstance(suffix_orig[0], dict) else None,
            'has_Activity': 'Activity' in suffix_orig[0] if suffix_orig and len(suffix_orig) > 0 and isinstance(suffix_orig[0], dict) else False
        })
        # #endregion
        
        if len(orig_tuple) >= 3:
            mean_pred_orig = orig_tuple[2]
            print(f"DEBUG: mean_pred_orig type: {type(mean_pred_orig)}")
            print(f"DEBUG: mean_pred_orig length: {len(mean_pred_orig) if mean_pred_orig else 'None'}")
            
            # Check ALL events, not just first
            if mean_pred_orig and len(mean_pred_orig) > 0:
                all_keys_in_all_events = []
                for i, event in enumerate(mean_pred_orig[:5]):  # Check first 5 events
                    if isinstance(event, dict):
                        all_keys_in_all_events.append(list(event.keys()))
                        print(f"DEBUG: Event {i} keys: {list(event.keys())}")
                        print(f"DEBUG: Event {i} full content: {event}")
                    else:
                        all_keys_in_all_events.append(None)
                        print(f"DEBUG: Event {i} is not a dict: {type(event)}")
            
            # #region agent log
            log_debug('debug-session', 'run1', 'B', 'compare_robustness_models.ipynb:40', 'Checking mean_pred keys', {
                'mean_pred_type': str(type(mean_pred_orig)),
                'mean_pred_len': len(mean_pred_orig) if mean_pred_orig else None,
                'first_event_keys': list(mean_pred_orig[0].keys()) if mean_pred_orig and len(mean_pred_orig) > 0 and isinstance(mean_pred_orig[0], dict) else None,
                'has_Activity': 'Activity' in mean_pred_orig[0] if mean_pred_orig and len(mean_pred_orig) > 0 and isinstance(mean_pred_orig[0], dict) else False,
                'all_event_keys': [list(e.keys()) if isinstance(e, dict) else None for e in mean_pred_orig[:3]] if mean_pred_orig else []
            })
            # #endregion
            
            if mean_pred_orig and len(mean_pred_orig) > 0:
                print(f"DEBUG: First event type: {type(mean_pred_orig[0])}")
                print(f"DEBUG: First event: {mean_pred_orig[0]}")
                if isinstance(mean_pred_orig[0], dict):
                    print(f"DEBUG: First event keys: {mean_pred_orig[0].keys()}")
                    
                    # #region agent log
                    log_debug('debug-session', 'run1', 'D', 'compare_robustness_models.ipynb:52', 'Checking Activity values', {
                        'Activity_value': mean_pred_orig[0].get('Activity'),
                        'Activity_is_none': mean_pred_orig[0].get('Activity') is None,
                        'all_keys': list(mean_pred_orig[0].keys())
                    })
                    # #endregion
        
        # Check robustness_metrics for activity info (Hypothesis E)
        if 'robustness_metrics' in first_entry:
            robustness_metrics = first_entry['robustness_metrics']
            # #region agent log
            log_debug('debug-session', 'run1', 'E', 'compare_robustness_models.ipynb:60', 'Checking robustness_metrics', {
                'has_mean_prediction': 'mean_prediction' in robustness_metrics if robustness_metrics else False,
                'mean_prediction_keys': list(robustness_metrics.get('mean_prediction', {}).keys()) if robustness_metrics and 'mean_prediction' in robustness_metrics else None
            })
            # #endregion
    
    print("="*80 + "\n")

# Extract mean predictions
comparisons = []
for idx, ((case_name, prefix_len), entry) in enumerate(first_100_items, 1):
    # Handle both old (4-tuple) and new (6-tuple) formats
    try:
        orig_tuple = entry['original']
        pert_tuple = entry['perturbed']
        
        if len(orig_tuple) == 6:
            # New format: (prefix, suffix, mean_pred, sampled, mean_rt, sampled_rt)
            mean_pred_orig = orig_tuple[2]
            mean_pred_pert = pert_tuple[2]
        else:
            # Old format: (prefix, suffix, mean_pred, sampled)
            mean_pred_orig = orig_tuple[2]
            mean_pred_pert = pert_tuple[2]
        
        # Extract activity/concept:name sequences for comparison
        # For Helpdesk, use 'Activity' as the default (not 'concept:name')
        concept_name = 'Activity'  # Default, adjust if needed
        
        # #region agent log
        if idx <= 3:  # Log first 3 entries
            log_debug('debug-session', 'run1', 'A', f'compare_robustness_models.ipynb:extract_{idx}', 'Starting extraction', {
                'case_name': case_name,
                'prefix_len': prefix_len,
                'mean_pred_orig_len': len(mean_pred_orig) if mean_pred_orig else None,
                'mean_pred_pert_len': len(mean_pred_pert) if mean_pred_pert else None
            })
        # #endregion
        
        # Better detection: check all events, not just the first
        if mean_pred_orig and len(mean_pred_orig) > 0:
            # Try to find concept_name in any event
            possible_names = ['Activity', 'concept:name', 'activity']
            found_key = None
            for event in mean_pred_orig:
                if isinstance(event, dict):
                    for name in possible_names:
                        if name in event and event[name] is not None:
                            found_key = name
                            break
                    if found_key:
                        break
            
            # #region agent log
            if idx <= 3:
                log_debug('debug-session', 'run1', 'A', f'compare_robustness_models.ipynb:detect_{idx}', 'Key detection result', {
                    'found_key': found_key,
                    'default_concept_name': 'Activity',
                    'first_event_keys': list(mean_pred_orig[0].keys()) if isinstance(mean_pred_orig[0], dict) else None,
                    'all_keys_in_first_3_events': [list(e.keys()) if isinstance(e, dict) else None for e in mean_pred_orig[:3]]
                })
            # #endregion
            
            if found_key:
                concept_name = found_key
            else:
                # If Activity not found, warn and try to use what's available
                if idx == 1:
                    print(f"⚠️  Activity not found in predictions. Available keys: {list(mean_pred_orig[0].keys()) if isinstance(mean_pred_orig[0], dict) else 'N/A'}")
        
        # Extract activities - handle None values
        orig_activities = []
        if mean_pred_orig:
            for event in mean_pred_orig:
                if isinstance(event, dict):
                    activity = event.get(concept_name)
                    if activity is not None:
                        orig_activities.append(activity)
                    else:
                        orig_activities.append('N/A')
                else:
                    orig_activities.append('N/A')
        else:
            orig_activities = ['N/A (empty)']
        
        pert_activities = []
        if mean_pred_pert:
            for event in mean_pred_pert:
                if isinstance(event, dict):
                    activity = event.get(concept_name)
                    if activity is not None:
                        pert_activities.append(activity)
                    else:
                        pert_activities.append('N/A')
                else:
                    pert_activities.append('N/A')
        else:
            pert_activities = ['N/A (empty)']
        
        # #region agent log
        if idx <= 3:
            log_debug('debug-session', 'run1', 'B', f'compare_robustness_models.ipynb:extract_result_{idx}', 'Extraction results', {
                'concept_name_used': concept_name,
                'orig_activities': orig_activities[:5],
                'pert_activities': pert_activities[:5],
                'orig_all_n_a': all(a == 'N/A' for a in orig_activities),
                'pert_all_n_a': all(a == 'N/A' for a in pert_activities),
                'orig_len': len(orig_activities),
                'pert_len': len(pert_activities)
            })
        # #endregion
        
        # Compare sequences
        are_equal = orig_activities == pert_activities
        orig_len = len(orig_activities)
        pert_len = len(pert_activities)
        
        comparisons.append({
            'idx': idx,
            'case': case_name,
            'prefix_len': prefix_len,
            'orig_activities': orig_activities,
            'pert_activities': pert_activities,
            'orig_len': orig_len,
            'pert_len': pert_len,
            'are_equal': are_equal
        })
        
    except (KeyError, IndexError, TypeError) as e:
        print(f"Error processing entry {idx} ({case_name}, {prefix_len}): {e}")
        import traceback
        traceback.print_exc()
        continue

# Print detailed comparison
print(f"\nTotal entries compared: {len(comparisons)}\n")
print(f"{'#':<5} {'Case':<20} {'Prefix':<8} {'Orig Len':<10} {'Pert Len':<10} {'Match':<8} {'Orig Activities':<40} {'Pert Activities':<40}")
print("-"*150)

for comp in comparisons[:100]:  # Print all comparisons
    match_str = "✓" if comp['are_equal'] else "✗"
    orig_str = " → ".join(str(a) for a in comp['orig_activities'][:5])  # Show first 5 activities
    if len(comp['orig_activities']) > 5:
        orig_str += "..."
    pert_str = " → ".join(str(a) for a in comp['pert_activities'][:5])
    if len(comp['pert_activities']) > 5:
        pert_str += "..."
    
    print(f"{comp['idx']:<5} {comp['case']:<20} {comp['prefix_len']:<8} {comp['orig_len']:<10} {comp['pert_len']:<10} {match_str:<8} {orig_str:<40} {pert_str:<40}")

# Summary statistics
matches = sum(1 for c in comparisons if c['are_equal'])
total = len(comparisons)
match_rate = (matches / total * 100) if total > 0 else 0

print("\n" + "="*80)
print(f"SUMMARY:")
print(f"  Total compared: {total}")
print(f"  Exact matches: {matches}")
print(f"  Mismatches: {total - matches}")
print(f"  Match rate: {match_rate:.2f}%")
print("="*80)

COMPARING MEAN PREDICTIONS: CLEAN vs PERTURBED (First 100)

DEBUG: Inspecting first entry: ('A', 1)
DEBUG: Entry keys: dict_keys(['original', 'perturbed', 'robustness_metrics', 'support_of_correct_prediction', 'nll_scores', 'wasserstein_distance', 'rouge_l_scores', 'chrf_scores', 'remaining_time_prediction_shift', 'prediction_shift_dls'])
DEBUG: Original tuple length: 6
DEBUG: mean_pred_orig type: <class 'list'>
DEBUG: mean_pred_orig length: 50
DEBUG: Event 0 keys: ['concept:name', 'case_elapsed_time']
DEBUG: Event 0 full content: {'concept:name': 'ER Registration', 'case_elapsed_time': array([386707.39555803])}
DEBUG: Event 1 keys: ['concept:name', 'case_elapsed_time']
DEBUG: Event 1 full content: {'concept:name': 'ER Registration', 'case_elapsed_time': array([387308.37764442])}
DEBUG: Event 2 keys: ['concept:name', 'case_elapsed_time']
DEBUG: Event 2 full content: {'concept:name': 'ER Registration', 'case_elapsed_time': array([387694.87814093])}
DEBUG: Event 3 keys: ['concept:name', 

In [8]:
# Print prefix activities for first 100 cases
print("="*80)
print("PREFIX ACTIVITIES FOR FIRST 100 CASES")
print("="*80)

# Get first 100 entries
first_100_items = list(U_ED_LSTM_results.items())[:100]

# Detect concept name from first entry
concept_name = 'Activity'  # Default
if first_100_items:
    first_key, first_entry = first_100_items[0]
    if 'original' in first_entry:
        orig_tuple = first_entry['original']
        if len(orig_tuple) > 0:
            prefix_orig = orig_tuple[0]
            if prefix_orig and len(prefix_orig) > 0:
                first_event = prefix_orig[0]
                if isinstance(first_event, dict):
                    # Try to find the concept name
                    possible_names = ['Activity', 'concept:name', 'activity']
                    for name in possible_names:
                        if name in first_event:
                            concept_name = name
                            break

print(f"\nUsing concept name: '{concept_name}'\n")
print(f"{'#':<5} {'Case Name':<25} {'Prefix Len':<12} {'Prefix Activities':<60}")
print("-"*100)

for idx, ((case_name, prefix_len), entry) in enumerate(first_100_items, 1):
    try:
        # Extract prefix from original tuple
        orig_tuple = entry['original']
        prefix = orig_tuple[0]  # Prefix is first element
        
        # Extract activities from prefix
        prefix_activities = []
        if prefix:
            for event in prefix:
                if isinstance(event, dict):
                    activity = event.get(concept_name)
                    if activity is not None:
                        prefix_activities.append(str(activity))
                    else:
                        prefix_activities.append('N/A')
                else:
                    prefix_activities.append('N/A')
        else:
            prefix_activities = ['N/A (empty)']
        
        # Format activities string (show first 10, truncate if longer)
        activities_str = " → ".join(prefix_activities[:10])
        if len(prefix_activities) > 10:
            activities_str += f" ... ({len(prefix_activities)} total)"
        
        print(f"{idx:<5} {case_name:<25} {prefix_len:<12} {activities_str:<60}")
        
    except (KeyError, IndexError, TypeError) as e:
        print(f"{idx:<5} {case_name:<25} {prefix_len:<12} ERROR: {e}")

print("\n" + "="*80)

PREFIX ACTIVITIES FOR FIRST 100 CASES

Using concept name: 'concept:name'

#     Case Name                 Prefix Len   Prefix Activities                                           
----------------------------------------------------------------------------------------------------
1     A                         1            ER Registration                                             
2     A                         2            ER Registration → Leucocytes                                
3     A                         3            ER Registration → Leucocytes → CRP                          
4     A                         4            ER Registration → Leucocytes → CRP → LacticAcid             
5     A                         5            ER Registration → Leucocytes → CRP → LacticAcid → ER Triage 
6     A                         6            ER Registration → Leucocytes → CRP → LacticAcid → ER Triage → ER Sepsis Triage
7     A                         7            ER Registration → L

In [ ]:
# Print ground truth suffixes for first 100 cases
print("="*80)
print("GROUND TRUTH SUFFIXES FOR FIRST 100 CASES")
print("="*80)

# Get first 100 entries
first_100_items = list(U_ED_LSTM_results.items())[:100]

# Detect concept name from first entry
concept_name = 'Activity'  # Default
if first_100_items:
    first_key, first_entry = first_100_items[0]
    if 'original' in first_entry:
        orig_tuple = first_entry['original']
        if len(orig_tuple) > 1:
            suffix_orig = orig_tuple[1]  # Suffix is second element
            if suffix_orig and len(suffix_orig) > 0:
                first_event = suffix_orig[0]
                if isinstance(first_event, dict):
                    # Try to find the concept name
                    possible_names = ['Activity', 'concept:name', 'activity']
                    for name in possible_names:
                        if name in first_event:
                            concept_name = name
                            break

print(f"\nUsing concept name: '{concept_name}'\n")
print(f"{'#':<5} {'Case Name':<25} {'Prefix Len':<12} {'Ground Truth Suffix Activities':<60}")
print("-"*100)

for idx, ((case_name, prefix_len), entry) in enumerate(first_100_items, 1):
    try:
        # Extract suffix from original tuple (ground truth)
        orig_tuple = entry['original']
        suffix = orig_tuple[1]  # Suffix is second element
        
        # Extract activities from suffix
        suffix_activities = []
        if suffix:
            for event in suffix:
                if isinstance(event, dict):
                    activity = event.get(concept_name)
                    if activity is not None:
                        suffix_activities.append(str(activity))
                    else:
                        suffix_activities.append('N/A')
                else:
                    suffix_activities.append('N/A')
        else:
            suffix_activities = ['N/A (empty)']
        
        # Format activities string (show first 10, truncate if longer)
        activities_str = " → ".join(suffix_activities[:10])
        if len(suffix_activities) > 10:
            activities_str += f" ... ({len(suffix_activities)} total)"
        
        print(f"{idx:<5} {case_name:<25} {prefix_len:<12} {activities_str:<60}")
        
    except (KeyError, IndexError, TypeError) as e:
        print(f"{idx:<5} {case_name:<25} {prefix_len:<12} ERROR: {e}")

print("\n" + "="*80)

In [ ]:
# Summary Statistics Table
import numpy as np

print("="*80)
print("SUMMARY COMPARISON")
print("="*80)
print(f"\n{'Metric':<30} {'Camargo LSTM':<20} {'Normal Model':<20} {'Difference':<15}")
print("-"*80)

# Calculate overall metrics from results
camargo_mean_metrics = [entry['robustness_metrics']['mean_prediction'] for entry in camargo_results.values() if 'robustness_metrics' in entry]
normal_mean_metrics = [entry['robustness_metrics']['mean_prediction'] for entry in U_ED_LSTM_results.values() if 'robustness_metrics' in entry]

camargo_prob_metrics = [entry['robustness_metrics'].get('probabilistic_prediction', {}) for entry in camargo_results.values() if 'robustness_metrics' in entry]
camargo_prob_metrics = [m for m in camargo_prob_metrics if m]
normal_prob_metrics = [entry['robustness_metrics'].get('probabilistic_prediction', {}) for entry in U_ED_LSTM_results.values() if 'robustness_metrics' in entry]
normal_prob_metrics = [m for m in normal_prob_metrics if m]

camargo_activity_match = np.mean([m['activity_sequence_match'] for m in camargo_mean_metrics])
normal_activity_match = np.mean([m['activity_sequence_match'] for m in normal_mean_metrics])
camargo_length_match = np.mean([m['length_match'] for m in camargo_mean_metrics])
normal_length_match = np.mean([m['length_match'] for m in normal_mean_metrics])

metrics_comparison = [
    ('Activity Match Rate', camargo_activity_match, normal_activity_match),
    ('Length Match Rate', camargo_length_match, normal_length_match),
]

# Add Top k metrics if available
if camargo_prob_metrics and normal_prob_metrics:
    camargo_topk = np.mean([m.get('top_k_activity_match_rate', 0.0) for m in camargo_prob_metrics])
    normal_topk = np.mean([m.get('top_k_activity_match_rate', 0.0) for m in normal_prob_metrics])
    metrics_comparison.append(
        ('Top k Activity Match Rate', camargo_topk, normal_topk)
    )

for metric_name, camargo_val, normal_val in metrics_comparison:
    diff = camargo_val - normal_val
    diff_str = f"{diff:+.4f}" if abs(diff) > 0.0001 else "≈ 0.0000"
    print(f"{metric_name:<30} {camargo_val:<20.4f} {normal_val:<20.4f} {diff_str:<15}")

print(f"\nTotal Evaluations:")
print(f"  Camargo LSTM: {len(camargo_results)}")
print(f"  Normal Model: {len(U_ED_LSTM_results)}")


In [ ]:
# Single comparison chart: Activity Sequence Match Rate by Prefix Length
import matplotlib as mpl

# Reset to defaults and apply clean styling similar to plot_res
mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,          # General font size
    'axes.titlesize': 10,    # Title font size
    'axes.labelsize': 9,     # Axis label size
    'xtick.labelsize': 8,    # X-axis tick labels
    'ytick.labelsize': 8,    # Y-axis tick labels
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,    # Legend font size
    'lines.linewidth': 1.2,  # Line width
    'lines.markersize': 5    # Marker size
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

# Get common prefix lengths
common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

# Create dictionaries for easy lookup
camargo_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['activity_match_rates']))
normal_dict = dict(zip(normal_data['prefix_lengths'], normal_data['activity_match_rates']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

# Extract values for common prefix lengths
camargo_values = [camargo_dict[p] for p in common_prefix_lengths]
normal_values = [normal_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

# Plot lines for activity match rate
line1 = ax1.plot(common_prefix_lengths, camargo_values, marker='o', 
                 linewidth=1.2, markersize=5, label='Camargo LSTM', 
                 color='blue', alpha=0.8)
line2 = ax1.plot(common_prefix_lengths, normal_values, marker='s', 
                 linewidth=1.2, markersize=5, label='U-ED-LSTM', 
                 color='orange', alpha=0.8)

# Create secondary y-axis for instance counts
ax2 = ax1.twinx()

# Plot instance counts as background bars
total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray', 
        width=0.6, label='Total instances')

# Style the axes
ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Activity Sequence Match Rate', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

# Set limits
ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

# Remove spines for cleaner look
for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

# Add grid
ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)

# Add reference line at 1.0
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

# Create combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right", 
           frameon=True, fontsize=8)

# Hide secondary y-axis ticks
ax2.set_yticks([])

# Bring primary plot to front
ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Remaining Time Prediction Shift by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_rt_shift_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['remaining_time_prediction_shift']))
normal_rt_shift_dict = dict(zip(normal_data['prefix_lengths'], normal_data['remaining_time_prediction_shift']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_rt_shift = [camargo_rt_shift_dict.get(p, 0.0) for p in common_prefix_lengths]
normal_rt_shift = [normal_rt_shift_dict.get(p, 0.0) for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict.get(p, 0) for p in common_prefix_lengths]
normal_counts = [normal_counts_dict.get(p, 0) for p in common_prefix_lengths]

# Plot lines for mean remaining time prediction shift
# ax1.plot(common_prefix_lengths, camargo_rt_shift, marker='o',
#          linewidth=1.2, markersize=5, label='Camargo LSTM',
#          color='blue', alpha=0.9)
ax1.plot(common_prefix_lengths, normal_rt_shift, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM',
         color='orange', alpha=0.9)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Remaining Time Prediction Shift', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Single comparison chart: Length Match Rate by Prefix Length
import matplotlib as mpl

# Reset to defaults and apply clean styling similar to plot_res
mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,          # General font size
    'axes.titlesize': 10,    # Title font size
    'axes.labelsize': 9,     # Axis label size
    'xtick.labelsize': 8,    # X-axis tick labels
    'ytick.labelsize': 8,    # Y-axis tick labels
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,    # Legend font size
    'lines.linewidth': 1.2,  # Line width
    'lines.markersize': 5    # Marker size
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

# Get common prefix lengths
common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

# Create dictionaries for easy lookup (using length_match_rates)
camargo_length_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['length_match_rates']))
normal_length_dict = dict(zip(normal_data['prefix_lengths'], normal_data['length_match_rates']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

# Extract values for common prefix lengths
camargo_length_values = [camargo_length_dict[p] for p in common_prefix_lengths]
normal_length_values = [normal_length_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

# Plot lines for length match rate
line1 = ax1.plot(common_prefix_lengths, camargo_length_values, marker='o', 
                 linewidth=1.2, markersize=5, label='Camargo LSTM', 
                 color='blue', alpha=0.8)
line2 = ax1.plot(common_prefix_lengths, normal_length_values, marker='s', 
                 linewidth=1.2, markersize=5, label='U-ED-LSTM', 
                 color='orange', alpha=0.8)

# Create secondary y-axis for instance counts
ax2 = ax1.twinx()

# Plot instance counts as background bars
total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray', 
        width=0.6, label='Total instances')

# Style the axes
ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Length Match Rate', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

# Set limits
ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

# Remove spines for cleaner look
for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

# Add grid
ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)

# Add reference line at 1.0
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

# Create combined legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right", 
           frameon=True, fontsize=8)

# Hide secondary y-axis ticks
ax2.set_yticks([])

# Bring primary plot to front
ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Single comparison chart: Clean DLS by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_clean_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['clean_dls']))
normal_clean_dict = dict(zip(normal_data['prefix_lengths'], normal_data['clean_dls']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_clean_values = [camargo_clean_dict[p] for p in common_prefix_lengths]
normal_clean_values = [normal_clean_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

ax1.plot(common_prefix_lengths, camargo_clean_values, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM',
         color='blue', alpha=0.8)
ax1.plot(common_prefix_lengths, normal_clean_values, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM',
         color='orange', alpha=0.8)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Clean DLS', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Single comparison chart: DLS drop under Perturbation by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_clean_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['relative_dls_drop']))
normal_clean_dict = dict(zip(normal_data['prefix_lengths'], normal_data['relative_dls_drop']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_clean_values = [camargo_clean_dict[p] for p in common_prefix_lengths]
normal_clean_values = [normal_clean_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

ax1.plot(common_prefix_lengths, camargo_clean_values, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM',
         color='blue', alpha=0.8)
ax1.plot(common_prefix_lengths, normal_clean_values, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM',
         color='orange', alpha=0.8)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('DLS drop under Perturbation', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Single comparison chart: Modal DLS on Clean Data by Prefix Length with IQR
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_modal_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['modal_clean_dls']))
normal_modal_dict = dict(zip(normal_data['prefix_lengths'], normal_data['modal_clean_dls']))
camargo_q25_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['clean_dls_q25']))
camargo_q75_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['clean_dls_q75']))
normal_q25_dict = dict(zip(normal_data['prefix_lengths'], normal_data['clean_dls_q25']))
normal_q75_dict = dict(zip(normal_data['prefix_lengths'], normal_data['clean_dls_q75']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_modal_values = [camargo_modal_dict[p] for p in common_prefix_lengths]
normal_modal_values = [normal_modal_dict[p] for p in common_prefix_lengths]
camargo_q25_values = [camargo_q25_dict[p] for p in common_prefix_lengths]
camargo_q75_values = [camargo_q75_dict[p] for p in common_prefix_lengths]
normal_q25_values = [normal_q25_dict[p] for p in common_prefix_lengths]
normal_q75_values = [normal_q75_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

# Plot IQR ranges (fill_between) first so they appear behind the lines
ax1.fill_between(common_prefix_lengths, camargo_q25_values, camargo_q75_values,
                 color='blue', alpha=0.15, label='Camargo LSTM IQR')
ax1.fill_between(common_prefix_lengths, normal_q25_values, normal_q75_values,
                 color='orange', alpha=0.15, label='U-ED-LSTM IQR')

# Plot modal DLS lines
ax1.plot(common_prefix_lengths, camargo_modal_values, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM',
         color='blue', alpha=0.8)
ax1.plot(common_prefix_lengths, normal_modal_values, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM',
         color='orange', alpha=0.8)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Modal DLS on Clean Data', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Single comparison chart: Modal DLS on Perturbed Data by Prefix Length with IQR
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_modal_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['modal_perturbed_dls']))
normal_modal_dict = dict(zip(normal_data['prefix_lengths'], normal_data['modal_perturbed_dls']))
camargo_q25_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['perturbed_dls_q25']))
camargo_q75_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['perturbed_dls_q75']))
normal_q25_dict = dict(zip(normal_data['prefix_lengths'], normal_data['perturbed_dls_q25']))
normal_q75_dict = dict(zip(normal_data['prefix_lengths'], normal_data['perturbed_dls_q75']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_modal_values = [camargo_modal_dict[p] for p in common_prefix_lengths]
normal_modal_values = [normal_modal_dict[p] for p in common_prefix_lengths]
camargo_q25_values = [camargo_q25_dict[p] for p in common_prefix_lengths]
camargo_q75_values = [camargo_q75_dict[p] for p in common_prefix_lengths]
normal_q25_values = [normal_q25_dict[p] for p in common_prefix_lengths]
normal_q75_values = [normal_q75_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

# Plot IQR ranges (fill_between) first so they appear behind the lines
ax1.fill_between(common_prefix_lengths, camargo_q25_values, camargo_q75_values,
                 color='blue', alpha=0.15, label='Camargo LSTM IQR')
ax1.fill_between(common_prefix_lengths, normal_q25_values, normal_q75_values,
                 color='orange', alpha=0.15, label='U-ED-LSTM IQR')

# Plot modal DLS lines
ax1.plot(common_prefix_lengths, camargo_modal_values, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM',
         color='blue', alpha=0.8)
ax1.plot(common_prefix_lengths, normal_modal_values, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM',
         color='orange', alpha=0.8)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Modal DLS on Perturbed Data', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Support of Correct Prediction by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_support_clean_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['support_clean']))
camargo_support_pert_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['support_perturbed']))
normal_support_clean_dict = dict(zip(normal_data['prefix_lengths'], normal_data['support_clean']))
normal_support_pert_dict = dict(zip(normal_data['prefix_lengths'], normal_data['support_perturbed']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_support_clean = [camargo_support_clean_dict[p] for p in common_prefix_lengths]
camargo_support_pert = [camargo_support_pert_dict[p] for p in common_prefix_lengths]
normal_support_clean = [normal_support_clean_dict[p] for p in common_prefix_lengths]
normal_support_pert = [normal_support_pert_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

ax1.plot(common_prefix_lengths, camargo_support_clean, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM (clean)',
         color='blue', alpha=0.9)
ax1.plot(common_prefix_lengths, camargo_support_pert, marker='o', linestyle='--',
         linewidth=1.2, markersize=5, label='Camargo LSTM (perturbed)',
         color='blue', alpha=0.6)
ax1.plot(common_prefix_lengths, normal_support_clean, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (clean)',
         color='orange', alpha=0.9)
ax1.plot(common_prefix_lengths, normal_support_pert, marker='s', linestyle='--',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (perturbed)',
         color='orange', alpha=0.6)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Support of Correct Prediction', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# ROUGE-L Score by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_rouge_clean_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['rouge_l_clean']))
camargo_rouge_pert_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['rouge_l_perturbed']))
normal_rouge_clean_dict = dict(zip(normal_data['prefix_lengths'], normal_data['rouge_l_clean']))
normal_rouge_pert_dict = dict(zip(normal_data['prefix_lengths'], normal_data['rouge_l_perturbed']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_rouge_clean = [camargo_rouge_clean_dict[p] for p in common_prefix_lengths]
camargo_rouge_pert = [camargo_rouge_pert_dict[p] for p in common_prefix_lengths]
normal_rouge_clean = [normal_rouge_clean_dict[p] for p in common_prefix_lengths]
normal_rouge_pert = [normal_rouge_pert_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

ax1.plot(common_prefix_lengths, camargo_rouge_clean, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM (clean)',
         color='blue', alpha=0.9)
ax1.plot(common_prefix_lengths, camargo_rouge_pert, marker='o', linestyle='--',
         linewidth=1.2, markersize=5, label='Camargo LSTM (perturbed)',
         color='blue', alpha=0.6)
ax1.plot(common_prefix_lengths, normal_rouge_clean, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (clean)',
         color='orange', alpha=0.9)
ax1.plot(common_prefix_lengths, normal_rouge_pert, marker='s', linestyle='--',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (perturbed)',
         color='orange', alpha=0.6)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('ROUGE-L Score', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# chrF Score by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_chrf_clean_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['chrf_clean']))
camargo_chrf_pert_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['chrf_perturbed']))
normal_chrf_clean_dict = dict(zip(normal_data['prefix_lengths'], normal_data['chrf_clean']))
normal_chrf_pert_dict = dict(zip(normal_data['prefix_lengths'], normal_data['chrf_perturbed']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_chrf_clean = [camargo_chrf_clean_dict[p] for p in common_prefix_lengths]
camargo_chrf_pert = [camargo_chrf_pert_dict[p] for p in common_prefix_lengths]
normal_chrf_clean = [normal_chrf_clean_dict[p] for p in common_prefix_lengths]
normal_chrf_pert = [normal_chrf_pert_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

ax1.plot(common_prefix_lengths, camargo_chrf_clean, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM (clean)',
         color='blue', alpha=0.9)
ax1.plot(common_prefix_lengths, camargo_chrf_pert, marker='o', linestyle='--',
         linewidth=1.2, markersize=5, label='Camargo LSTM (perturbed)',
         color='blue', alpha=0.6)
ax1.plot(common_prefix_lengths, normal_chrf_clean, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (clean)',
         color='orange', alpha=0.9)
ax1.plot(common_prefix_lengths, normal_chrf_pert, marker='s', linestyle='--',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (perturbed)',
         color='orange', alpha=0.6)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('chrF Score', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_ylim(0, 1.05)
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, linewidth=0.7)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Negative Log Likelihood (NLL) by Prefix Length
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

common_prefix_lengths = sorted(set(camargo_data['prefix_lengths']) & set(normal_data['prefix_lengths']))

camargo_nll_clean_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['nll_clean']))
camargo_nll_pert_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['nll_perturbed']))
normal_nll_clean_dict = dict(zip(normal_data['prefix_lengths'], normal_data['nll_clean']))
normal_nll_pert_dict = dict(zip(normal_data['prefix_lengths'], normal_data['nll_perturbed']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

camargo_nll_clean = [camargo_nll_clean_dict[p] for p in common_prefix_lengths]
camargo_nll_pert = [camargo_nll_pert_dict[p] for p in common_prefix_lengths]
normal_nll_clean = [normal_nll_clean_dict[p] for p in common_prefix_lengths]
normal_nll_pert = [normal_nll_pert_dict[p] for p in common_prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in common_prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in common_prefix_lengths]

ax1.plot(common_prefix_lengths, camargo_nll_clean, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM (clean)',
         color='blue', alpha=0.9)
ax1.plot(common_prefix_lengths, camargo_nll_pert, marker='o', linestyle='--',
         linewidth=1.2, markersize=5, label='Camargo LSTM (perturbed)',
         color='blue', alpha=0.6)
ax1.plot(common_prefix_lengths, normal_nll_clean, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (clean)',
         color='orange', alpha=0.9)
ax1.plot(common_prefix_lengths, normal_nll_pert, marker='s', linestyle='--',
         linewidth=1.2, markersize=5, label='U-ED-LSTM (perturbed)',
         color='orange', alpha=0.6)

ax2 = ax1.twinx()

total_counts = [camargo_counts[i] + normal_counts[i] for i in range(len(common_prefix_lengths))]
ax2.bar(common_prefix_lengths, total_counts, alpha=0.15, color='gray',
        width=0.6, label='Total instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Negative Log Likelihood', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

# For NLL, lower is better, so we don't set a fixed upper limit
# Instead, let matplotlib auto-scale based on the data
ax1.set_xlim(left=min(common_prefix_lengths) - 0.5, right=max(common_prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Wasserstein Distance by Prefix Length - Camargo LSTM
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

camargo_wasserstein_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['wasserstein_distance']))
camargo_counts_dict = dict(zip(camargo_data['prefix_lengths'], camargo_data['sample_counts']))

prefix_lengths = sorted(camargo_data['prefix_lengths'])
camargo_wasserstein = [camargo_wasserstein_dict[p] for p in prefix_lengths]
camargo_counts = [camargo_counts_dict[p] for p in prefix_lengths]

ax1.plot(prefix_lengths, camargo_wasserstein, marker='o',
         linewidth=1.2, markersize=5, label='Camargo LSTM',
         color='blue', alpha=0.9)

ax2 = ax1.twinx()

ax2.bar(prefix_lengths, camargo_counts, alpha=0.15, color='gray',
        width=0.6, label='Instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Wasserstein Distance', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_xlim(left=min(prefix_lengths) - 0.5, right=max(prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# Wasserstein Distance by Prefix Length - U-ED-LSTM
import matplotlib as mpl

mpl.rcdefaults()
plt.rcParams.update({
    'font.size': 8,
    'axes.titlesize': 10,
    'axes.labelsize': 9,
    'xtick.labelsize': 8,
    'ytick.labelsize': 8,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'legend.fontsize': 8,
    'lines.linewidth': 1.2,
    'lines.markersize': 5
})

fig, ax1 = plt.subplots(figsize=(6, 4), dpi=100)

normal_wasserstein_dict = dict(zip(normal_data['prefix_lengths'], normal_data['wasserstein_distance']))
normal_counts_dict = dict(zip(normal_data['prefix_lengths'], normal_data['sample_counts']))

prefix_lengths = sorted(normal_data['prefix_lengths'])
normal_wasserstein = [normal_wasserstein_dict[p] for p in prefix_lengths]
normal_counts = [normal_counts_dict[p] for p in prefix_lengths]

ax1.plot(prefix_lengths, normal_wasserstein, marker='s',
         linewidth=1.2, markersize=5, label='U-ED-LSTM',
         color='orange', alpha=0.9)

ax2 = ax1.twinx()

ax2.bar(prefix_lengths, normal_counts, alpha=0.15, color='gray',
        width=0.6, label='Instances')

ax1.set_xlabel('prefix len', labelpad=0.5)
ax1.set_ylabel('Wasserstein Distance', labelpad=0.5)
ax2.set_ylabel('instances', labelpad=0.5)

ax1.set_xlim(left=min(prefix_lengths) - 0.5, right=max(prefix_lengths) + 0.5)
ax2.set_ylim(bottom=0)

for spine in ax1.spines.values():
    spine.set_visible(False)
for spine in ax2.spines.values():
    spine.set_visible(False)

ax1.grid(True, alpha=0.3, axis='y', linestyle='--', linewidth=0.5)

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc="upper right",
           frameon=True, fontsize=8)

ax2.set_yticks([])

ax1.set_zorder(2)
ax2.set_zorder(1)
ax1.patch.set_visible(False)

plt.tight_layout()
plt.show()


## Adversarial Sample Analysis

Identify the most effective adversarial samples - those that caused the biggest shift in model predictions.

In [ ]:
from adversarial_sample_selector import get_best_adversarial_prefixes

# Analyze the most effective adversarial samples for U-ED-LSTM
print("="*80)
print("ANALYZING U-ED-LSTM MODEL")
print("="*80)

best_adversarial_samples = get_best_adversarial_prefixes(
    U_ED_LSTM_results, 
    top_n=10,  # Show top 10 most adversarial samples
    concept_name='Activity'
)

# The function returns a list of tuples with detailed information
# Each tuple contains: (case_name, prefix_len, dls, prefix_orig, prefix_pert, mean_orig, mean_pert, perturbations)

In [ ]:
U_ED_LSTM_results